# TP Attaques adversariales

Dans ce TP, nous allons effectuer des attaques adversariales sur un réseau de neurones entraîné pour identifier des chiffres MNIST. Une attaque adversariale consiste à trouver une perturbation très petite des entrées du réseau (une modification de l'image) qui permet de tromper le réseau de neurones. Puis, nous renforcerons l'apprentissage pour tenter de rendre le réseau de neurones robuste à ces attaques.

Dans ce TP, des cellules seront laissées à trous, il faudra les compléter suivant les consignes. Elles seront identifiées par le mot **Exercice**. Les **Vérifications** seront effectuées principalement par vous-mêmes, sur la bonne convergence des algorithmes ou leur bon fonctionnement.

Ci-dessous, on importe les bibliothèques qui seront utiles.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow as tf

## Mise en place des données

Le code ci-dessous charge les données MNIST.

In [ ]:
#NE PAS MODIFIER

(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

**Exercice** : Normalisez les données d'entrées en les divisant par 255 et passez les données de sortie sous forme catégorielle (one hot encoding, en utilisant keras.utils.to_categorical). Si le réseau de neurones que vous utilisez est convolutif, n'oubliez pas d'ajouter un axe sur les données d'entrée représentant le nombre de canaux (1 seul canal ici pour les images en niveau de gris).

In [ ]:
#TO DO

X_train =

X_test =

Y_train_cat =

Y_test_cat =

## Modèle Keras

**Exercice** : Importez le modèle pré-entraîné que vous souhaitez (ou entraînez un nouveau modèle) et applez le "my_model". Vous pouvez utiliser le modèle fourni en exemple. Pour importer un modèle sauvegardé, vous pouvez utiliser keras.models.load_model.

In [ ]:
my_model = #TO DO



**Exercice** : Affichez la structure de votre modèle avec my_model.summary()

In [ ]:
#TO DO

**Vérification** : Pour l'instant, il suffit qu'il n'y ait pas d'erreur.

## Prédictions avec le modèle

**Exercice** : Effectuez la prédiction sur le jeu de test.

In [ ]:
Y_pred_test = #TO DO

**Exercice** : Extrayez les labels prédits par votre réseau, qui correspondent aux classes avec la plus grande probabilité. La fonction np.argmax vous sera utile, à appliquer sur le bon "axis".

In [ ]:
Y_test_pred_lab = #TO DO

**Exercice** : Calculez l'accuracy sur le jeu de test et notez-la bien.

In [ ]:
#TO DO

Le code ci-dessous vous permet de visualiser quelques résultats pris au hasard sur la base de test.

In [ ]:
r = np.random.randint(X_test.shape[0])

figure = plt.figure(figsize = (16,9))

ax1 = plt.subplot(121)
ax1.imshow(X_test[r,:,:,0],cmap = "hot")
plt.title("Prédiction du réseau : " + str(Y_test_pred_lab[r]) + "\n Vraie valeur : " + str(Y_test[r]))

ax2 = plt.subplot(122)
ax2.bar(np.arange(10),height = Y_pred_test[r],tick_label = np.arange(10))
plt.xlabel("Valeur")
plt.ylabel("Output du réseau")


**Exercice** : Réutilisez le code d'affichage ci-dessus, mais pour afficher aléatoirement des erreurs. La fonction np.where vous sera utile pour localiser les erreurs.

In [ ]:
#TO DO

## Attaques adversariales

Nous allons maintenant appliquer des attaques adversariales aux données d'entrées. Pour rappel, effectuer une attaque adversariale sur un exemple $x$ pour un réseau de neurones $f_{\theta}$ consiste à trouver une perturbation $\delta$ qui maximise la fonction de coût $L$ sur l'exemple perturbé $x + \delta$, par rapport à la réponse attendue $y$, sous la contrainte que $\delta$ soit petite (de norme inférieure à $\varepsilon$). Mathématiquement :

\begin{equation}
\delta^{*} = \underset{\lVert \delta \rVert < \varepsilon}{\mathrm{argmax}}~L(f_{\theta}(x + \delta),y)
\end{equation}

où $\delta^{*}$ serait ici une attaque optimale.

### Calcul du gradient de la loss function par rapport aux entrées

Les méthodes d'attaque classiques consistent à calculer le gradient de la fonction de coût par rapport aux entrées et à "remonter" ce gradient.

Il est donc nécessaire dans un premier temps de calculer ce gradient pour chaque entrée $x$ :

\begin{equation}
\nabla_x L(f_\theta(x),y)
\end{equation}

**Exercice** : La bibliothèque Tensorflow a les outils nécessaires pour calculer le gradient d'une quantité par rapport à une autre quantité, tant que les liens entre ces deux quantités sont des fonctions Tensorflow. Une fonction tensorflow peut être par exemple tf.math.log, tf.math.exp, une addition simple de deux quantités, un produit, une puissance... ou même un réseau de neurones tensorflow (ou keras) !
L'exercice consiste alors à compléter la fonction suivante qui, pour une entrée (ou un ensemble d'entrées), un label donné et un modèle donné renvoie le gradient de la fonction de coût (ici Categorical CrossEntropy) par rapport aux entrées.

Pour compléter :
- Dans la partie "with tf.GradientTape() as tape", nous traquons la variable inputs (c'est relativement à cette variable que nous calculons le gradient). Cela se fait à partir de la commande tape.watch(inputs).
- Il faut maintenant compléter le code, toujours en respectant l'indentation de gradient.tape, pour obtenir la valeur de la loss associée à ces entrées et ces labels :
    - Effectuez la prédiction sur les inputs (utilisez model(inputs) et non pas model.predict(inputs) !, car model(inputs) est une opération "tensorflow" et model.predict(inputs) est une opération numpy et elle n'est pas traquée par le gradient).
    - Une fois la prédiction effectuée, calculez la loss entre cette prédiction et le label attendu à l'aide de loss_cross, défini au début de la cellule.
- Sortez ensuite de l'indentation du GradientTape et retournez le gradient de la loss ainsi calculée par rapport aux inputs à l'aide de tape.gradient(loss,inputs).
    

In [ ]:
loss_cross = tf.keras.losses.CategoricalCrossentropy()

def Calculgradient(inputs, label, model):
    
    label = tf.reshape(label,(label.shape[0],10))
    
    with tf.GradientTape() as tape:
        
        tape.watch(inputs)
        
        prediction = #TO DO
        loss = #TO DO
    
    gradient = #TO DO
    
    return gradient


### Attaque FGSM (Fast Gradient Signed Method)

L'attaque FGSM consiste à ne considérer que le signe de chaque composante gradient pour trouver la direction de l'attaque. C'est une attaque très rapide (une seule itération) et dans ce cas, la perturbation est simplement :

\begin{equation}
\delta = \lambda~\mathrm{sign}( \nabla_x L(f_\theta(x),y))
\end{equation}

Ici $\lambda$ est un coefficient qui permet de choisir l'amplitude de la perturbation. Plus $\lambda$ est petit, plus l'attaque sera indiscernable mais potentiellement inefficace.

**Exercice** : La fonction ci-dessous vise à calculer la perturbation à appliquer sur chaque attaque en suivant la méthode FGSM : cette perturbation est caculée pour des entrées inputs, avec des sorties attendues notées label, pour un modèle noté model et avec un coefficent $\lambda$ noté lambd. Utilisez la fonction Calculgradient définie précédemment et la fonction tf.math.sign pourrait être utile.

Ici les inputs ne sont pas considérés comme des tenseurs tf et les labels ne sont pas catégoricals, d'où les deux premières lignes pré-remplies.

In [ ]:
def delta_fgsm(inputs, label, model, lambd):

    inputs_tf = tf.constant(inputs,dtype = "float32")
    lab = keras.utils.to_categorical(label,num_classes=10)

    gradient = #TO DO
    delta = #TO DO

    return delta

**Exercice** : Complétez le code ci-dessous pour effectuer la prédiction de la perturbation sur l'ensemble des données de test à partir de delta_fgsm. On choisit ici une amplitude de perturbation égale à 1 et nous modifierons l'amplitude par la suite lorsque nous l'appliquerons aux images.

In [ ]:
alpha = 1.

delta = #TO DO

pred_cur = my_model.predict(X_test)

**Exercice** : Appliquez la perturbation delta calculée dans la cellule précédente à X_test. Le terme $\lambd$ permet de régler son amplitude (la perturbation sera alors $\lambda \times \delta$). Une fois l'image adverse calculée, ses limites peuvent dépasser les intensités limites des images MNIST (entre 0 et 1 avec notre normalisation). Utilisez la fonction np.clip pour remettre les images entre 0 et 1. Enfin effectuez la prédiction du modèle sur les images adverses.

Exécutez aussi la cellule suivante pour voir l'effet de la perturbation sur vos données et sur la prédiction.

Essayez de modifier la valeur de lambd pour voir l'effet de l'amplitude de la perturbation.

In [ ]:
lambd = 0.2

#TO DO

x_adv =

pred_adv = my_model.predict(x_adv)

In [ ]:
#NE PAS MODIFIER

i = np.random.randint(np.shape(X_test)[0])

figure = plt.figure(figsize = (16,9))

ax1 = plt.subplot(221)
ax1.imshow(X_test[i,:,:,0],cmap = "Greys_r")
plt.title("Original image")

ax2 = plt.subplot(222)
ax2.bar(np.arange(10),height = pred_cur[i],tick_label = np.arange(10))
plt.xlabel("Valeur")
plt.ylabel("Output du réseau")
plt.title("Prédiction du réseau : " + str(np.argmax(pred_cur,axis = 1)[i]) + "\n Vraie valeur : " + str(Y_test[i]))

ax3 = plt.subplot(223)
ax3.imshow(x_adv[i,:,:,0],cmap = "Greys_r")
plt.title("Adversarial image")

ax4 = plt.subplot(224)
ax4.bar(np.arange(10),height = pred_adv[i],tick_label = np.arange(10))
plt.xlabel("Valeur")
plt.ylabel("Output du réseau")
plt.title("Prédiction du réseau : " + str(np.argmax(pred_adv,axis = 1)[i]) + "\n Vraie valeur : " + str(Y_test[i]))

plt.tight_layout()

**Exercice** : Pour les différentes valeurs de lambd ci-dessous, calculez l'accuracy de votre réseau de neurones sur le jeu de test attaqué. Complétez la boucle suivante, à chaque itération :
- Calculez les adversaires du jeu de test selon l'amplitude lambd_cur
- Effectuez la prédiction du modèle sur ces adversaires
- Calculez l'accuracy du modèle et stockez-la dans accuracy_tab

In [ ]:
lambd_vec = np.linspace(0,1,50)

accuracy_tab = lambd_vec*0.

for i in range(np.size(lambd_vec)):
    
    lambd_cur = lambd_vec[i]
    
    #TO DO ....
    
    accuracy_tab[i] = ...
    
    print(i)

Ci-dessous, voici l'évolution de l'accuracy du modèle en fonction de la valeur de lambda.

In [ ]:
#NE PAS MODIFIER

figure = plt.figure(figsize = (10,8))

plt.plot(lambd_vec,accuracy_tab,label = "Adversarial images",color = "red")
plt.plot(lambd_vec,accuracy_tab[0] + accuracy_tab*0,label = "Original images (baseline)",color = "blue")
plt.xlabel("Amplitude")
plt.ylabel("Accuracy")
plt.legend()

### Adversarial learning

Nous allons mettre en place une procédure d'entraînement adverse. Cela consiste à chaque époque d'entraînement à calculer les exmples adverses et effectuer l'entraînement du modèle sur ces exemples adverses.

**Exercice** : Complétez la fonction suivante afin qu'à chaque itération :
- vous calculiez les exemples adverses sur la base d'entraînement (n'oubliez pas le clip entre 0 et 1)
- vous effectuiez l'apprentissage du modèle sur ces nouvelles données d'entraînement adverse. Une seule époque à chaque itération de la boucle devrait suffire.

Si l'apprentissage est trop lent, n'hésitez pas à l'arrêter au moment où vous le souhaitez.

In [ ]:
N_epochs = 100

lambd = 0.2

for i in range(N_epochs):
    
    #TO DO ....
    
    my_model.fit ....

**Exercice** : Procédez à l'évaluation de votre nouveau modèle : calculez d'abord les nouvelles perturbations pour alpha = 1 puis faites varier l'amplitude lambd dans la boucle pour appliquer les perturbations avec ces différentes amplitudes et calculer l'accuracy à cahque amplitude que vous stockerez dans accuracy_tab_robuste.

In [ ]:
lambd_vec = np.linspace(0,1,50)

accuracy_tab_robuste = lambd_vec*0.

alpha = 1.

delta = #TO DO

for i in range(np.size(lambd_vec)):
    
    lambd_cur = lambd_vec[i]
    
    #TO DO ....
    
    accuracy_tab_robuste[i] = #TO DO
    
    print(i)

Visualisez ci-dessous le gain au niveau de l'accuracy sur les exemples adverses.

In [ ]:
#NE PAS MODIFIER

figure = plt.figure(figsize = (10,8))

plt.plot(lambd_vec,accuracy_tab[0] + accuracy_tab*0,label = "Original images on previous model (baseline)",color = "blue")
plt.plot(lambd_vec,accuracy_tab,label = "Adversarial images on previous model (baseline)",color = "red")
plt.plot(lambd_vec,accuracy_tab_robuste[0] + accuracy_tab*0,label = "Original images on new model",color = "purple")
plt.plot(lambd_vec,accuracy_tab_robuste,label = "Adversarial images on new model",color = "orange")

plt.xlabel("Amplitude")
plt.ylabel("Accuracy")
plt.legend()

## Pour la suite

Vous pouvez recommencer cet exercice avec de nouvelles données. Vous pouvez aussi changer l'algorithme d'attaque qui consistait ici en une seule itération de gradient, mais vous pouvez aussi appliquer une descente de gradient en plusieurs étapes.